In [1]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns 
import re
from ast import literal_eval
warnings.filterwarnings('ignore')
print('pandas:'+pd.__version__)
print('seaborn:'+sns.__version__)  
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer 
import nltk
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
from nltk.corpus import stopwords 
from sklearn.multiclass import OneVsRestClassifier
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from datetime import datetime
import sqlite3
from sqlite3 import Error
import uuid
from time import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

pandas:1.0.5
seaborn:0.10.1


[nltk_data] Downloading package stopwords to /home/igor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
results = []
resultsLogisticRegression = []


# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

id_transacao = dt_string#str(uuid.uuid4())
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

def save_metrics(conn, project):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO metrics( classificador, score_f1,
    precision,
    train_time,
    test_time,
    transacao)
    VALUES(?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid

def salvar_dado(valor):
    database = r"../data/metricsDB.sqlite"
    print (id)
    # create a database connection
    conn = create_connection(database)
    with conn:
        # create a new project
#         linha = ['SGD Classifier',  0.4503092535879421,  1.9019618034362793,  0.012923479080200195, id]
#         metrica = ('Cool App with SQLite & Python', '2015-01-01', '2015-01-30');
        valor.append(id_transacao)
        print(valor)
        metrica = save_metrics( conn,valor) 
    print (id)
        

now = 2020-10-02 21:54:29.147387
date and time = 02/10/2020 21:54:29


In [3]:
tmdb = pd.read_csv('../../../dados/tmdb_5000_movies.csv')

In [4]:
tmdb['genres'] = tmdb['genres'].fillna('[]')\
.apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
tmdb.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [5]:
tmdb['genres']=tmdb['genres'].apply(lambda x: '|'.join(x))
tmdb.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,Action|Adventure|Fantasy|Science Fiction,http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,Adventure|Fantasy|Action,http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,Action|Adventure|Crime,http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [6]:
generos = tmdb.genres.str.get_dummies()
generos.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [7]:
tmdb.isnull().sum()


budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [8]:
tmdb['overview'].fillna('', inplace=True)
tmdb.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   0
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [9]:

# dados_dos_filmes.drop(['budget','homepage','id','keywords','original_language',
#                             'original_title','popularity','production_companies','production_countries',
#                            'release_date','revenue','runtime','spoken_languages','status','tagline',
#                            'title','vote_average','vote_count', 'overview','genres' ], axis=1)

In [10]:
generos = tmdb.genres.str.get_dummies()
generos.head()

#CONCATENA OS DOIS DATAFRAMES
dados_dos_filmes = pd.concat([tmdb, generos], axis=1)
dados_dos_filmes.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,237000000,Action|Adventure|Fantasy|Science Fiction,http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,0,0,0,0,0,1,0,0,0,0
1,300000000,Adventure|Fantasy|Action,http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,0,0,0,0,0,0,0,0,0,0
2,245000000,Action|Adventure|Crime,http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,0,0,0,0,0,0,0,0,0,0
3,250000000,Action|Crime|Drama|Thriller,http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,0,0,0,0,0,0,0,1,0,0
4,260000000,Action|Adventure|Science Fiction,http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,0,0,0,0,0,1,0,0,0,0


In [11]:
# df_genres =dados_dos_filmes.drop(['budget','homepage','id','keywords','original_language',
#                             'original_title','popularity','production_companies','production_countries',
#                            'release_date','revenue','runtime','spoken_languages','status','tagline',
#                            'title','vote_average','vote_count', 'overview','genres' ], axis=1)
# df_genres

In [12]:
df_genres =dados_dos_filmes.drop(['budget','homepage','id','keywords','original_language',
                            'original_title','popularity','production_companies','production_countries',
                           'release_date','revenue','runtime','spoken_languages','status','tagline',
                           'title','vote_average','vote_count', 'overview','genres' ], axis=1)
counts = []
categories = list(df_genres.columns)
for i in categories:
    counts.append((i, df_genres[i].sum()))
df_stats = pd.DataFrame(counts, columns=['genre', '#movies'])
df_stats

,genre,#movies
0,Action,1154
1,Adventure,790
2,Animation,234
3,Comedy,1722
4,Crime,696
5,Documentary,110
6,Drama,2297
7,Family,513
8,Fantasy,424
9,Foreign,34


In [13]:
# dados_dos_filmes.drop(['budget','homepage','id','keywords','original_language',
#                             'original_title','popularity','production_companies','production_countries',
# #                            'release_date','revenue','runtime','spoken_languages','status','tagline',
#                            'title','vote_average','vote_count', 'overview','genres' ], axis=1)

In [14]:

# split the data, leave 1/3 out for testing
data_x = dados_dos_filmes[['overview']]
data_y = dados_dos_filmes.drop(['budget','homepage','id','keywords','original_language',
                            'original_title','popularity','production_companies','production_countries',
                           'release_date','revenue','runtime','spoken_languages','status','tagline',
                           'title','vote_average','vote_count', 'overview','genres' ], axis=1)
# stratified_split = StratifiedShuffleSplit(n_splits=2, test_size=0.33)
xtrain, xval, ytrain, yval = model_selection.train_test_split(data_x, data_y, test_size=0.33, random_state=512)
# transform matrix of plots into lists to pass to a TfidfVectorizer
train_x = [x[0].strip() for x in xtrain.values.tolist()]
test_x = [x[0].strip() for x in xval.values.tolist()] 

In [15]:
from sklearn.ensemble import ExtraTreesClassifier
nltk.download('stopwords') 
words = stopwords.words("english") 
pipeline = Pipeline([
('tfidf', TfidfVectorizer(stop_words=words)),
('clf',  OneVsRestClassifier(ExtraTreesClassifier()) ),
    
])
parameters = { 
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__max_features': [50,100, 1000,5000,10000,15000],
    "clf__estimator__n_estimators": [10,15,20],
    "clf__estimator__class_weight": ['balanced','balanced_subsample', None],
    'clf__estimator__max_depth':[8,12,15,20],
}

[nltk_data] Downloading package stopwords to /home/igor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:

grid_search_tune = GridSearchCV(
pipeline, parameters, cv=2, n_jobs=2, verbose=3)
grid_search_tune.fit(train_x, ytrain )  
print
print("Best parameters set:")
print (grid_search_tune.best_estimator_.steps)

Fitting 2 folds for each of 648 candidates, totalling 1296 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    8.0s
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed:   45.4s
[Parallel(n_jobs=2)]: Done 284 tasks      | elapsed:  1.9min
[Parallel(n_jobs=2)]: Done 508 tasks      | elapsed:  3.6min
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:  5.6min
[Parallel(n_jobs=2)]: Done 1148 tasks      | elapsed:  8.3min
[Parallel(n_jobs=2)]: Done 1296 out of 1296 | elapsed:  9.8min finished


Best parameters set:
[('tfidf', TfidfVectorizer(max_features=5000,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])), ('clf', OneVsRestClassifier(estimator=ExtraTreesClassifier(class_weight='balanced_subsample',
                                                   max_depth=20,
                                                   n_estimators=20)))]


In [17]:

clf  = grid_search_tune.best_estimator_
classes=categories
print('_' * 120)
print("Training: ")
print(clf)
t0 = time()  
train_time = time() - t0
print("Treino: %0.3fs" % train_time)

t0 = time()
pred = clf.predict(test_x)
test_time = time() - t0
print("testes:  %0.3fs" % test_time)

score = f1_score(yval, pred, average="micro")
precision = precision_score(yval, pred, average='micro')
print("f1_score:   %0.3f" % score)

print('\nAccuracy: {:.2f}\n'.format(accuracy_score(yval, pred)))
print('\nPrecisao: {:.2f}\n'.format(precision)) 

print("Relatório de Classificação:")
print(classification_report(yval, pred,
#                                             labels=[1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
                                        target_names=classes))

# print("confusion matrix:") 
# print(confusion_matrix(yvalb2.argmax(axis=1), pred.argmax(axis=1)))




clf_descr = 'ExtraTreesClassifier-semtratamento-tmdb'#str(clf).split('(')[0]

test_time=55555    
results.append([clf_descr, score,precision, train_time, test_time])  
resultsLogisticRegression.append([clf_descr, score,precision, train_time, test_time])  
salvar_dado([clf_descr, score,precision, train_time, test_time])


________________________________________________________________________________________________________________________
Training: 
Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=5000,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('clf',
                 OneVsRestClassifier(estimator=ExtraTreesClassifier(class_weight='balanced_subsample',
                                         

# Fazendo nova predição com upsample

In [18]:
tmdb = pd.read_csv('../../../dados/tmdb_5000_movies.csv')

In [19]:
tmdb['genres'] = tmdb['genres'].fillna('[]')\
.apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
tmdb.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [20]:
tmdb['overview'].fillna('', inplace=True)
tmdb.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   0
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [21]:
overview_df = tmdb[["overview","genres"]]
filmes_balanceados = overview_df.copy(deep=True)

In [22]:
lst_col = 'genres'

filmes_balanceados2 = pd.DataFrame({
      col:np.repeat(filmes_balanceados[col].values, filmes_balanceados[lst_col].str.len())
      for col in filmes_balanceados.columns.drop(lst_col)}
    ).assign(**{
    lst_col:np.concatenate(filmes_balanceados[lst_col].values)})[filmes_balanceados.columns]

In [23]:
filmes_balanceados2['genres'].value_counts()

Drama              2297
Comedy             1722
Thriller           1274
Action             1154
Romance             894
Adventure           790
Crime               696
Science Fiction     535
Horror              519
Family              513
Fantasy             424
Mystery             348
Animation           234
History             197
Music               185
War                 144
Documentary         110
Western              82
Foreign              34
TV Movie              8
Name: genres, dtype: int64

In [24]:
df_Drama = filmes_balanceados2[filmes_balanceados2['genres']=='Drama']
df_Comedy  = filmes_balanceados2[filmes_balanceados2['genres']=='Comedy']
df_Thriller   = filmes_balanceados2[filmes_balanceados2['genres']=='Thriller']
df_Action  = filmes_balanceados2[filmes_balanceados2['genres']=='Action']
df_Romance   = filmes_balanceados2[filmes_balanceados2['genres']=='Romance']
df_Adventure = filmes_balanceados2[filmes_balanceados2['genres']=='Adventure']
df_Crime    = filmes_balanceados2[filmes_balanceados2['genres']=='Crime']
df_Science_Fiction = filmes_balanceados2[filmes_balanceados2['genres']=='Science Fiction']
df_Horror     = filmes_balanceados2[filmes_balanceados2['genres']=='Horror']
df_Family           = filmes_balanceados2[filmes_balanceados2['genres']=='Family']
df_Fantasy    = filmes_balanceados2[filmes_balanceados2['genres']=='Fantasy']
df_Mystery      = filmes_balanceados2[filmes_balanceados2['genres']=='Mystery']
df_Animation   = filmes_balanceados2[filmes_balanceados2['genres']=='Animation']
df_History       = filmes_balanceados2[filmes_balanceados2['genres']=='History']
df_Music         = filmes_balanceados2[filmes_balanceados2['genres']=='Music']
df_War        = filmes_balanceados2[filmes_balanceados2['genres']=='War']
df_Documentary  = filmes_balanceados2[filmes_balanceados2['genres']=='Documentary']
df_Western     = filmes_balanceados2[filmes_balanceados2['genres']=='Western']
df_Foreign     = filmes_balanceados2[filmes_balanceados2['genres']=='Foreign']
df_TV_Movie  = filmes_balanceados2[filmes_balanceados2['genres']=='TV Movie']

In [25]:
from sklearn.utils import resample
df_Comedy  = resample(df_Comedy,
                      replace=True,
                      n_samples=2297,
                      random_state=123)
df_Thriller   = resample(df_Thriller,
                         replace=True,
                         n_samples=2297,
                         random_state=123)
df_Action  = resample(df_Action,
                      replace=True,
                      n_samples=2297,
                      random_state=123)
df_Romance   = resample(df_Romance,
                        replace=True,
                        n_samples=2297,
                        random_state=123)
df_Adventure= resample(df_Adventure, 
                       replace=True,  
                       n_samples=2297,    
                       random_state=123)
df_Crime   = resample(df_Crime,
                      replace=True,     
                      n_samples=2297,    
                      random_state=123)
df_Science_Fiction = resample(df_Science_Fiction,
                              replace=True, 
                              n_samples=2297,
                              random_state=123)
df_Horror     = resample(df_Horror, 
                         replace=True,     
                         n_samples=2297,    
                         random_state=123)
df_Family          = resample(df_Family, 
                              replace=True,
                              n_samples=2297,  
                              random_state=123)
df_Fantasy   = resample(df_Fantasy, 
                                 replace=True,     
                                 n_samples=2297,    
                                 random_state=123)
df_Mystery      = resample(df_Mystery,
                           replace=True,     
                           n_samples=2297,   
                           random_state=123)
df_Animation  = resample(df_Animation, 
                         replace=True, 
                         n_samples=2297, 
                         random_state=123)
df_History    = resample(df_History, 
                         replace=True, 
                         n_samples=2297,  
                         random_state=123)
df_Music      = resample(df_Music,
                         replace=True,  
                         n_samples=2297, 
                         random_state=123)
df_War       = resample(df_War, 
                        replace=True,   
                        n_samples=2297,    
                        random_state=123)
df_Documentary= resample(df_Documentary, 
                         replace=True,
                         n_samples=2297,    
                         random_state=123)
df_Western    = resample(df_Western, 
                         replace=True,     
                         n_samples=2297,    
                         random_state=123)
df_Foreign   = resample(df_Foreign, 
                        replace=True,
                        n_samples=2297,  
                        random_state=123)
df_TV_Movie = resample(df_TV_Movie, 
                       replace=True, 
                       n_samples=2297,
                       random_state=123)

In [26]:
df_upsampled = pd.concat([df_Drama, df_Comedy], axis=0 ) 
df_upsampled = pd.concat([df_upsampled, df_Adventure], axis=0 )


df_upsampled = pd.concat([df_upsampled, df_Thriller ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Action ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Romance ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Crime ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Science_Fiction ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Horror ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Family ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Fantasy ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Mystery ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Animation ]
                         )
df_upsampled = pd.concat([df_upsampled, df_History ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Music ]
                         )
df_upsampled = pd.concat([df_upsampled, df_War ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Documentary ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Western ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Foreign ]
                         )
df_upsampled = pd.concat([df_upsampled, df_TV_Movie ]
                         )

In [27]:
df_upsampled.reset_index()

,index,overview,genres
0,12,Following the death of District Attorney Harve...,Drama
1,74,"In 1933 New York, an overly ambitious movie pr...",Drama
2,76,"84 years later, a 101-year-old woman named Ros...",Drama
3,139,Life for former United Nations investigator Ge...,Drama
4,153,An adaptation of F. Scott Fitzgerald's Long Is...,Drama
...,...,...,...
45935,12158,"""Signed, Sealed, Delivered"" introduces a dedic...",TV Movie
45936,9934,"Troy (Zac Efron), the popular captain of the b...",TV Movie
45937,8903,A young woman finds out that her mother worked...,TV Movie
45938,10413,The continued Westward journey of settlers Mis...,TV Movie


In [28]:
df_upsampled['genres'].value_counts()

Crime              2297
TV Movie           2297
Fantasy            2297
Science Fiction    2297
Adventure          2297
Family             2297
Romance            2297
History            2297
Documentary        2297
Drama              2297
Horror             2297
Action             2297
Mystery            2297
Music              2297
Foreign            2297
Western            2297
War                2297
Animation          2297
Thriller           2297
Comedy             2297
Name: genres, dtype: int64

In [29]:
df_upsampled['genres'].unique()

array(['Drama', 'Comedy', 'Adventure', 'Thriller', 'Action', 'Romance',
       'Crime', 'Science Fiction', 'Horror', 'Family', 'Fantasy',
       'Mystery', 'Animation', 'History', 'Music', 'War', 'Documentary',
       'Western', 'Foreign', 'TV Movie'], dtype=object)

In [30]:
from sklearn.preprocessing import MultiLabelBinarizer
multilabel_binarizer3 = MultiLabelBinarizer()
df_upsampled['genres'] = df_upsampled['genres'].map(lambda x: x.split(','))
y =multilabel_binarizer3.fit_transform( df_upsampled['genres'] )
tfidf_vectorizer3 = TfidfVectorizer(max_df=0.7, max_features=25000)
# from sklearn import model_selection
xtrain, xval, ytrain, yval = model_selection.train_test_split(df_upsampled['overview'].values, y, test_size=0.2, random_state=512)
xtrain_tfidf = tfidf_vectorizer3.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer3.transform(xval)
train_x = xtrain_tfidf
test_x = xval_tfidf

In [31]:
multilabel_binarizer3.classes_

array(['Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
       'TV Movie', 'Thriller', 'War', 'Western'], dtype=object)

In [32]:
  
from sklearn.ensemble import ExtraTreesClassifier
nltk.download('stopwords') 
words = stopwords.words("english") 
pipeline = Pipeline([
('tfidf', TfidfVectorizer(stop_words=words)),
('clf',  OneVsRestClassifier(ExtraTreesClassifier()) ),
    
])
parameters = {  
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__max_features': [100, 1000,5000,10000,15000],
    "clf__estimator__n_estimators": [10,15,20],
    "clf__estimator__class_weight": ['balanced','balanced_subsample', None],
    'clf__estimator__max_depth':[8,12,15,20],
} 
# rfc = OneVsRestClassifier(ExtraTreesClassifier())
# parameters = {'n_estimators':[10,15,20],  
#              "class_weight": ['balanced','balanced_subsample', None],
#              'max_depth':[3,5,8]}
    
# clf = GridSearchCV(svc, parameters, cv=2, n_jobs=2, verbose=3) 
 

[nltk_data] Downloading package stopwords to /home/igor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
multilabel_binarizer3.classes_

array(['Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
       'TV Movie', 'Thriller', 'War', 'Western'], dtype=object)

In [34]:
train_x.shape

(36752, 20685)

In [35]:
ytrain.shape

(36752, 20)

In [36]:


classes=multilabel_binarizer3.classes_
print('_' * 120)


t0 = time()  
##############
print("Training: ")
# grid_search_tune = GridSearchCV(
# rfc, parameters, cv=2, n_jobs=2, verbose=3)
grid_search_tune = GridSearchCV(
pipeline, parameters, cv=2, n_jobs=2, verbose=3)

grid_search_tune.fit(train_x, ytrain )  

clf  = grid_search_tune.best_estimator_
print(clf)
print 
############################
train_time = time() - t0
print("Treino: %0.3fs" % train_time)

t0 = time()
pred = clf.predict(test_x)
test_time = time() - t0
print("testes:  %0.3fs" % test_time)

score = f1_score(yval, pred, average="micro")
precision = precision_score(yval, pred, average='micro') 
print("f1_score:   %0.3f" % score)

print('\nAccuracy: {:.2f}\n'.format(accuracy_score(yval, pred)))
print('\nPrecisao: {:.2f}\n'.format(precision)) 


print('Micro Precision: {:.2f}'.format(precision_score(yval, pred, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(yval, pred, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(yval, pred, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(yval, pred, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(yval, pred, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(yval, pred, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(yval, pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(yval, pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(yval, pred, average='weighted')))

print("Relatório de Classificação:")
print(classification_report(yval, pred,
#                                             labels=[1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
                                        target_names=classes))

# print("confusion matrix:") 
# print(confusion_matrix(yvalb2.argmax(axis=1), pred.argmax(axis=1)))


if hasattr(clf, 'coef_'):
    print("dimensionality: %d" % clf.coef_.shape[1])
    print("density: %f" % density(clf.coef_))

clf_descr = 'ExtraTreesClassifier-upsampled-tf-idf-tmdb'#str(clf).split('(')[0]

test_time=55555    
results.append([clf_descr, score,precision, train_time, test_time])  
resultsLogisticRegression.append([clf_descr, score,precision, train_time, test_time])  
salvar_dado([clf_descr, score,precision, train_time, test_time])


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


________________________________________________________________________________________________________________________
Training: 
Fitting 2 folds for each of 540 candidates, totalling 1080 fits


[Parallel(n_jobs=2)]: Done 252 tasks      | elapsed:    2.1s
[Parallel(n_jobs=2)]: Done 1080 out of 1080 | elapsed:    8.7s finished


AttributeError: lower not found

In [71]:
 clf.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'tfidf', 'clf', 'tfidf__analyzer', 'tfidf__binary', 'tfidf__decode_error', 'tfidf__dtype', 'tfidf__encoding', 'tfidf__input', 'tfidf__lowercase', 'tfidf__max_df', 'tfidf__max_features', 'tfidf__min_df', 'tfidf__ngram_range', 'tfidf__norm', 'tfidf__preprocessor', 'tfidf__smooth_idf', 'tfidf__stop_words', 'tfidf__strip_accents', 'tfidf__sublinear_tf', 'tfidf__token_pattern', 'tfidf__tokenizer', 'tfidf__use_idf', 'tfidf__vocabulary', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_scor

# MovieLens

In [3]:
movies = pd.read_csv('../../../dados/movielens/filmes_com_overview.csv')

In [4]:
# movies = movies['genres != (no genres listed)']:
movies.drop(movies[movies.genres == '(no genres listed)'].index, inplace=True)

In [5]:
movies['genres'] = movies.genres.apply(lambda x: x.split('|'))

In [6]:
movies.head(3)

,movieId,title,genres,overview
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","Led by Woody, Andy's toys live happily in his ..."
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",When siblings Judy and Peter discover an encha...
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",A family wedding reignites the ancient feud be...


In [7]:
 (movies['genres'].values == '').sum()  

0

In [8]:
movies['overview'].fillna('', inplace=True)
movies.isnull().sum()

movieId     0
title       0
genres      0
overview    0
dtype: int64

In [9]:
movies = movies[movies['genres'] != 'chi Fade to Black Kimi no Na o Yobu) (2008)']
movies = movies[movies['genres'] != ' Battle Force (1978)']
movies = movies[movies['genres'] != 'epe&#351']
movies = movies[movies['genres'] != '(no genres listed)']
movies = movies[movies['genres'] != 'Gate the Movie: The Burden of Déjà vu (2013)']
movies['genres'].value_counts()

[Drama]                                                   8869
[Comedy]                                                  5422
[Documentary]                                             4573
[Comedy, Drama]                                           2338
[Drama, Romance]                                          2088
                                                          ... 
[Film-Noir, Horror]                                          1
[Adventure, Animation, Crime, Mystery]                       1
[Action, Crime, Thriller, Western]                           1
[Adventure, Animation, Comedy, Drama, Fantasy, Sci-Fi]       1
[Action, Crime, Musical, Romance]                            1
Name: genres, Length: 1625, dtype: int64

In [51]:
# generos = movies.genres.str.get_dummies()
# generos.head()

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
# dados_dos_filmes = pd.concat([movies, generos], axis=1)
# dados_dos_filmes.head()

,movieId,title,genres,overview,Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"Led by Woody, Andy's toys live happily in his ...",0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,When siblings Judy and Peter discover an encha...,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,A family wedding reignites the ancient feud be...,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"Cheated on, mistreated and stepped on, the wom...",0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,Just when George Banks has recovered from his ...,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# dados_dos_filmes = dados_dos_filmes[dados_dos_filmes['genres'] != 'chi Fade to Black Kimi no Na o Yobu) (2008)']
# dados_dos_filmes = dados_dos_filmes[dados_dos_filmes['genres'] != ' Battle Force (1978)']
# dados_dos_filmes = dados_dos_filmes[dados_dos_filmes['genres'] != 'epe&#351']
# dados_dos_filmes = dados_dos_filmes[dados_dos_filmes['genres'] != '(no genres listed)']
# dados_dos_filmes = dados_dos_filmes[dados_dos_filmes['genres'] != 'Gate the Movie: The Burden of Déjà vu (2013)']
# dados_dos_filmes['genres'].value_counts()

In [11]:
# dados_dos_filmes = dados_dos_filmes[dados_dos_filmes['genres'] != 'genres']
# dados_dos_filmes = dados_dos_filmes[dados_dos_filmes['genres'] != 'Gate the Movie: The Burden of Déjà vu (2013)']
# dados_dos_filmes.columns
# df_genres.columns

In [12]:
# dados_dos_filmes.shape

In [13]:

# df_genres =dados_dos_filmes.drop(['movieId','title', 'overview'  ], axis=1)
# counts = []
# categories = list(df_genres.columns)
# categories = categories[1:]
# for i in categories:
#     counts.append((i, df_genres[i].sum()))
# df_stats = pd.DataFrame(counts, columns=['genre', '#movies'])
# df_stats

In [14]:
title_df = movies[["overview","genres","title"]]

In [15]:
filmes_balanceados = title_df.copy(deep=True)

In [16]:
lst_col = 'genres'

filmes_balanceados2 = pd.DataFrame({
      col:np.repeat(filmes_balanceados[col].values, filmes_balanceados[lst_col].str.len())
      for col in filmes_balanceados.columns.drop(lst_col)}
    ).assign(**{
    lst_col:np.concatenate(filmes_balanceados[lst_col].values)})[filmes_balanceados.columns]

In [17]:
title_df.head(3)

,overview,genres,title
0,"Led by Woody, Andy's toys live happily in his ...","[Adventure, Animation, Children, Comedy, Fantasy]",Toy Story (1995)
1,When siblings Judy and Peter discover an encha...,"[Adventure, Children, Fantasy]",Jumanji (1995)
2,A family wedding reignites the ancient feud be...,"[Comedy, Romance]",Grumpier Old Men (1995)


In [18]:
filmes_balanceados2.tail(3)

,overview,genres,title
105092,Italian costume drama about convicted women in...,Action,Women of Devil's Island (1962)
105093,Italian costume drama about convicted women in...,Adventure,Women of Devil's Island (1962)
105094,Italian costume drama about convicted women in...,Drama,Women of Devil's Island (1962)


In [19]:
filmes_balanceados2['genres'].value_counts()

Drama                                           25112
Comedy                                          16440
Thriller                                         8569
Romance                                          7602
Action                                           7242
Horror                                           5935
Documentary                                      5410
Crime                                            5215
Adventure                                        4054
Sci-Fi                                           3528
Mystery                                          2863
Children                                         2859
Animation                                        2824
Fantasy                                          2644
War                                              1832
Western                                          1378
Musical                                          1039
Film-Noir                                         352
IMAX                        

In [28]:
 filmes_balanceados2 = filmes_balanceados2[filmes_balanceados2['genres'] != 'chi Fade to Black Kimi no Na o Yobu) (2008)']
 filmes_balanceados2 = filmes_balanceados2[filmes_balanceados2['genres'] != ' Battle Force (1978)']
 filmes_balanceados2 = filmes_balanceados2[filmes_balanceados2['genres'] != 'epe&#351']
 filmes_balanceados2 = filmes_balanceados2[filmes_balanceados2['genres'] != '(no genres listed)']
 filmes_balanceados2 = filmes_balanceados2[filmes_balanceados2['genres'] != 'Gate the Movie: The Burden of Déjà vu (2013)']
 filmes_balanceados2['genres'].value_counts()

Drama          25112
Comedy         16440
Thriller        8569
Romance         7602
Action          7242
Horror          5935
Documentary     5410
Crime           5215
Adventure       4054
Sci-Fi          3528
Mystery         2863
Children        2859
Animation       2824
Fantasy         2644
War             1832
Western         1378
Musical         1039
Film-Noir        352
IMAX             193
Name: genres, dtype: int64

In [29]:
df_Drama = filmes_balanceados2[filmes_balanceados2['genres']=='Drama']
df_Comedy  = filmes_balanceados2[filmes_balanceados2['genres']=='Comedy']
df_Thriller   = filmes_balanceados2[filmes_balanceados2['genres']=='Thriller']
df_Romance   = filmes_balanceados2[filmes_balanceados2['genres']=='Romance']
df_Action  = filmes_balanceados2[filmes_balanceados2['genres']=='Action']
df_Horror     = filmes_balanceados2[filmes_balanceados2['genres']=='Horror']
df_Documentary  = filmes_balanceados2[filmes_balanceados2['genres']=='Documentary'] 
df_Crime    = filmes_balanceados2[filmes_balanceados2['genres']=='Crime']
df_Adventure = filmes_balanceados2[filmes_balanceados2['genres']=='Adventure']
df_SciFi = filmes_balanceados2[filmes_balanceados2['genres']=='Sci-Fi']
df_Mystery      = filmes_balanceados2[filmes_balanceados2['genres']=='Mystery']
df_Children    = filmes_balanceados2[filmes_balanceados2['genres']=='Children']
df_Animation   = filmes_balanceados2[filmes_balanceados2['genres']=='Animation']
df_Fantasy    = filmes_balanceados2[filmes_balanceados2['genres']=='Fantasy']
df_War        = filmes_balanceados2[filmes_balanceados2['genres']=='War']
df_Western     = filmes_balanceados2[filmes_balanceados2['genres']=='Western']
df_Musical         = filmes_balanceados2[filmes_balanceados2['genres']=='Musical'] 
df_FilmNoir         = filmes_balanceados2[filmes_balanceados2['genres']=='Film-Noir'] 
df_IMAX    = filmes_balanceados2[filmes_balanceados2['genres']=='IMAX']

In [34]:
df_Comedy  = resample(df_Comedy,
                      replace=True,
                      n_samples=25112,
                      random_state=123)


df_Thriller   = resample(df_Thriller,
                         replace=True,
                         n_samples=25112,
                         random_state=123)
df_Romance   = resample(df_Romance,
                        replace=True,
                        n_samples=25112,
                        random_state=123)
df_Action  = resample(df_Action,
                      replace=True,
                      n_samples=25112,
                      random_state=123)
df_Horror     = resample(df_Horror, 
                         replace=True,     
                         n_samples=25112,    
                         random_state=123)
df_Documentary= resample(df_Documentary, 
                         replace=True,
                         n_samples=25112,    
                         random_state=123)
df_Crime   = resample(df_Crime,
                      replace=True,     
                      n_samples=25112,    
                      random_state=123)
df_Adventure= resample(df_Adventure, 
                       replace=True,  
                       n_samples=25112,    
                       random_state=123)
df_SciFi = resample(df_SciFi,
                              replace=True, 
                              n_samples=25112,
                              random_state=123)
df_Mystery      = resample(df_Mystery,
                           replace=True,     
                           n_samples=25112,   
                           random_state=123)
df_Children          = resample(df_Children, 
                              replace=True,
                              n_samples=25112,  
                              random_state=123)
df_Animation  = resample(df_Animation, 
                         replace=True, 
                         n_samples=25112, 
                         random_state=123)
df_Fantasy   = resample(df_Fantasy, 
                                 replace=True,     
                                 n_samples=25112,    
                                 random_state=123)
df_War       = resample(df_War, 
                        replace=True,   
                        n_samples=25112,    
                        random_state=123)
df_Western    = resample(df_Western, 
                         replace=True,     
                         n_samples=25112,    
                         random_state=123)
df_Musical      = resample(df_Musical,
                         replace=True,  
                         n_samples=25112, 
                         random_state=123)
df_FilmNoir    = resample(df_FilmNoir, 
                         replace=True, 
                         n_samples=25112,  
                         random_state=123)
df_IMAX   = resample(df_IMAX, 
                        replace=True,
                        n_samples=25112,  
                        random_state=123)


 

In [35]:
df_upsampled = pd.concat([df_Drama, df_Comedy], axis=0 )
df_upsampled = pd.concat([df_upsampled, df_Thriller ])
df_upsampled = pd.concat([df_upsampled, df_Romance ] )
df_upsampled = pd.concat([df_upsampled, df_Action ]  )
df_upsampled = pd.concat([df_upsampled, df_Horror ] )
df_upsampled = pd.concat([df_upsampled, df_Documentary ]  )
df_upsampled = pd.concat([df_upsampled, df_Crime ] )
df_upsampled = pd.concat([df_upsampled, df_Adventure], axis=0 )
df_upsampled = pd.concat([df_upsampled, df_SciFi ] )
df_upsampled = pd.concat([df_upsampled, df_Mystery ] )
df_upsampled = pd.concat([df_upsampled, df_Children ]  )
df_upsampled = pd.concat([df_upsampled, df_Animation ]  )
df_upsampled = pd.concat([df_upsampled, df_Fantasy ] )
df_upsampled = pd.concat([df_upsampled, df_War ]  )
df_upsampled = pd.concat([df_upsampled, df_Western ] )
df_upsampled = pd.concat([df_upsampled, df_Musical ] )

df_upsampled = pd.concat([df_upsampled, df_FilmNoir ] )
df_upsampled = pd.concat([df_upsampled, df_IMAX ]  ) 

In [36]:
df_upsampled.reset_index()

,index,overview,genres,title
0,11,"Cheated on, mistreated and stepped on, the wom...",Drama,Waiting to Exhale (1995)
1,26,"Widowed U.S. president Andrew Shepherd, one of...",Drama,"American President, The (1995)"
2,33,A look at President Richard M. Nixon—a man car...,Drama,Nixon (1995)
3,38,"In early-1970s Las Vegas, low-level mobster Sa...",Drama,Casino (1995)
4,39,"Rich Mr. Dashwood dies, leaving his second wif...",Drama,Sense and Sensibility (1995)
...,...,...,...,...
477123,8666,How to create the imagination of movement for ...,IMAX,Thrill Ride: The Science of Fun (1997)
477124,8674,Africa the Serengeti takes you on an extraordi...,IMAX,Africa: The Serengeti (1994)
477125,39987,"In 1942, Henan Province was devastated by the ...",IMAX,Back to 1942 (2012)
477126,34070,Star race car Lightning McQueen and his pal Ma...,IMAX,Cars 2 (2011)


In [37]:
df_upsampled['genres'].value_counts()

War            25112
Crime          25112
Fantasy        25112
Romance        25112
Horror         25112
Musical        25112
Action         25112
Drama          25112
Film-Noir      25112
Animation      25112
Western        25112
Children       25112
Documentary    25112
Mystery        25112
Adventure      25112
Comedy         25112
Sci-Fi         25112
IMAX           25112
Thriller       25112
Name: genres, dtype: int64

In [ ]:
PAROU AQUI

In [56]:
 # split the data, leave 1/3 out for testing
data_x = dados_dos_filmes[['overview']]
data_y = dados_dos_filmes.drop(['movieId', 'title', 'genres', 'overview'], axis=1)
# stratified_split = StratifiedShuffleSplit(n_splits=2, test_size=0.33)
xtrain, xval, ytrain, yval = model_selection.train_test_split(data_x, data_y, test_size=0.33, random_state=512)
# transform matrix of plots into lists to pass to a TfidfVectorizer
train_x = [x[0].strip() for x in xtrain.values.tolist()]
test_x = [x[0].strip() for x in xval.values.tolist()] 



In [106]:
data_y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56077 entries, 0 to 60788
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Action       56077 non-null  int64
 1   Adventure    56077 non-null  int64
 2   Animation    56077 non-null  int64
 3   Children     56077 non-null  int64
 4   Comedy       56077 non-null  int64
 5   Crime        56077 non-null  int64
 6   Documentary  56077 non-null  int64
 7   Drama        56077 non-null  int64
 8   Fantasy      56077 non-null  int64
 9   Film-Noir    56077 non-null  int64
 10  Horror       56077 non-null  int64
 11  IMAX         56077 non-null  int64
 12  Musical      56077 non-null  int64
 13  Mystery      56077 non-null  int64
 14  Romance      56077 non-null  int64
 15  Sci-Fi       56077 non-null  int64
 16  Thriller     56077 non-null  int64
 17  War          56077 non-null  int64
 18  Western      56077 non-null  int64
dtypes: int64(19)
memory usage: 8.6 MB


In [57]:
from sklearn.ensemble import RandomForestClassifier
nltk.download('stopwords') 
words = stopwords.words("english") 
pipeline = Pipeline([
('tfidf', TfidfVectorizer(stop_words=words)),
('rfc', ExtraTreesClassifier() ),
    
])
parameters = {
# 'tfidf__max_df': (0.5, 0.65, 0.75),
# 'tfidf__min_df': (0.15, 0.30, 0.35),
'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
"rfc__n_estimators": [10,15,20],
"rfc__class_weight": ['balanced','balanced_subsample', None],
'rfc__max_depth':[20,25,30],
}

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:

grid_search_tune = GridSearchCV(
pipeline, parameters, cv=2, n_jobs=2, verbose=3)
grid_search_tune.fit(train_x, ytrain )  
print
print("Best parameters set:")
print (grid_search_tune.best_estimator_.steps)

Fitting 2 folds for each of 162 candidates, totalling 324 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:  2.5min
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed: 10.8min
[Parallel(n_jobs=2)]: Done 284 tasks      | elapsed: 24.5min
[Parallel(n_jobs=2)]: Done 324 out of 324 | elapsed: 28.6min finished


Best parameters set:
[('tfidf', TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])), ('rfc', ExtraTreesClassifier(max_depth=25, n_estimators=10))]


In [104]:
len(grid_search_tune.best_estimator_.classes_)

20

In [107]:

clf  = grid_search_tune.best_estimator_
classes=categories
classes=[ 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
print('_' * 120)
print("Training: ")
print(clf)
t0 = time()  
train_time = time() - t0
print("Treino: %0.3fs" % train_time)

t0 = time()
pred = clf.predict(test_x)
test_time = time() - t0
print("testes:  %0.3fs" % test_time)

score = f1_score(yval, pred, average="micro")
precision = precision_score(yval, pred, average='micro')
print("f1_score:   %0.3f" % score)

print('\nAccuracy: {:.2f}\n'.format(accuracy_score(yval, pred)))
print('\nPrecisao: {:.2f}\n'.format(precision)) 

print("Relatório de Classificação:")
print(classification_report(yval, pred,
#                                             labels=[1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
                                        ))

# print("confusion matrix:") 
# print(confusion_matrix(yvalb2.argmax(axis=1), pred.argmax(axis=1)))


if hasattr(clf, 'coef_'):
    print("dimensionality: %d" % clf.coef_.shape[1])
    print("density: %f" % density(clf.coef_))

clf_descr = 'LogisticRegression-upsampled'#str(clf).split('(')[0]

test_time=55555    
results.append([clf_descr, score,precision, train_time, test_time])  
resultsLogisticRegression.append([clf_descr, score,precision, train_time, test_time])  
salvar_dado([clf_descr, score,precision, train_time, test_time])


________________________________________________________________________________________________________________________
Training: 
ExtraTreesClassifier(max_depth=8, n_estimators=10)
Treino: 0.000s
testes:  0.160s
f1_score:   0.062

Accuracy: 0.03


Precisao: 0.98

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       432
           1       0.00      0.00      0.00       446
           2       0.00      0.00      0.00       446
           3       0.00      0.00      0.00       471
           4       0.00      0.00      0.00       449
           5       0.00      0.00      0.00       496
           6       0.00      0.00      0.00       466
           7       0.00      0.00      0.00       483
           8       0.00      0.00      0.00       447
           9       0.00      0.00      0.00       447
          10       0.00      0.00      0.00       424
          11       0.00      0.00      0.00       472
   

# Refazer predição com resample

In [133]:
movies = pd.read_csv('../../../dados/movielens/filmes_com_overview.csv')
# movies = movies['genres != (no genres listed)']:
movies.drop(movies[movies.genres == '(no genres listed)'].index, inplace=True)
movies['genres'].value_counts()

Drama                                       8869
Comedy                                      5422
Documentary                                 4573
Comedy|Drama                                2338
Drama|Romance                               2088
                                            ... 
Adventure|Romance|War|Western                  1
Action|Children|Fantasy|Musical                1
Fantasy|Mystery|Sci-Fi|War                     1
Action|Adventure|Children|Comedy|Romance       1
Fantasy|Mystery|Sci-Fi|Thriller                1
Name: genres, Length: 1625, dtype: int64

In [134]:
movies['overview'].fillna('', inplace=True)
movies.isnull().sum()

movieId     0
title       0
genres      0
overview    0
dtype: int64

In [135]:
movies = movies[movies['genres'] != 'chi Fade to Black Kimi no Na o Yobu) (2008)']
movies = movies[movies['genres'] != ' Battle Force (1978)']
movies = movies[movies['genres'] != 'epe&#351']
movies = movies[movies['genres'] != '(no genres listed)']
movies = movies[movies['genres'] != 'Gate the Movie: The Burden of Déjà vu (2013)']
movies['genres'].value_counts()

Drama                                       8869
Comedy                                      5422
Documentary                                 4573
Comedy|Drama                                2338
Drama|Romance                               2088
                                            ... 
Action|Children|Fantasy|Musical                1
Fantasy|Mystery|Sci-Fi|War                     1
Action|Adventure|Children|Comedy|Romance       1
Action|Crime|Drama|Film-Noir|Thriller          1
Action|Adventure|Children|Comedy|Mystery       1
Name: genres, Length: 1621, dtype: int64

In [136]:
movies.head()

,movieId,title,genres,overview
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"Led by Woody, Andy's toys live happily in his ..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,When siblings Judy and Peter discover an encha...
2,3,Grumpier Old Men (1995),Comedy|Romance,A family wedding reignites the ancient feud be...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"Cheated on, mistreated and stepped on, the wom..."
4,5,Father of the Bride Part II (1995),Comedy,Just when George Banks has recovered from his ...


In [119]:
(movies['genres'].values == '').sum()  


0

In [137]:
movies['genres'] = movies['genres'].map(lambda x: x.split('|'))
movies.head()

,movieId,title,genres,overview
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","Led by Woody, Andy's toys live happily in his ..."
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",When siblings Judy and Peter discover an encha...
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",A family wedding reignites the ancient feud be...
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]","Cheated on, mistreated and stepped on, the wom..."
4,5,Father of the Bride Part II (1995),[Comedy],Just when George Banks has recovered from his ...


In [138]:
overview_df = movies[["overview","genres"]]
filmes_balanceados = overview_df.copy(deep=True)

In [139]:
lst_col = 'genres'

filmes_balanceados2 = pd.DataFrame({
      col:np.repeat(filmes_balanceados[col].values, filmes_balanceados[lst_col].str.len())
      for col in filmes_balanceados.columns.drop(lst_col)}
    ).assign(**{
    lst_col:np.concatenate(filmes_balanceados[lst_col].values)})[filmes_balanceados.columns]
 

In [140]:
df_Drama = filmes_balanceados2[filmes_balanceados2['genres']=='Drama']
df_Comedy  = filmes_balanceados2[filmes_balanceados2['genres']=='Comedy']
df_Thriller   = filmes_balanceados2[filmes_balanceados2['genres']=='Thriller']
df_Romance   = filmes_balanceados2[filmes_balanceados2['genres']=='Romance']
df_Action  = filmes_balanceados2[filmes_balanceados2['genres']=='Action']
df_Horror     = filmes_balanceados2[filmes_balanceados2['genres']=='Horror']
df_Documentary  = filmes_balanceados2[filmes_balanceados2['genres']=='Documentary']
df_Crime    = filmes_balanceados2[filmes_balanceados2['genres']=='Crime']
df_Adventure = filmes_balanceados2[filmes_balanceados2['genres']=='Adventure'] 
df_Mystery      = filmes_balanceados2[filmes_balanceados2['genres']=='Mystery'] 
df_Animation   = filmes_balanceados2[filmes_balanceados2['genres']=='Animation']
df_Fantasy    = filmes_balanceados2[filmes_balanceados2['genres']=='Fantasy'] 
df_War        = filmes_balanceados2[filmes_balanceados2['genres']=='War']
df_Western     = filmes_balanceados2[filmes_balanceados2['genres']=='Western']
df_Music         = filmes_balanceados2[filmes_balanceados2['genres']=='Musical']

df_Children           = filmes_balanceados2[filmes_balanceados2['genres']=='Children']
df_SciFi = filmes_balanceados2[filmes_balanceados2['genres']=='Sci-Fi']
df_FilmNoir           = filmes_balanceados2[filmes_balanceados2['genres']=='Film-Noir']
df_IMAX     = filmes_balanceados2[filmes_balanceados2['genres']=='IMAX']


In [141]:
df_Drama.shape

(25112, 2)

In [142]:
df_Comedy  = resample(df_Comedy,
                      replace=True,
                      n_samples=25112,
                      random_state=0)
df_Thriller   = resample(df_Thriller,
                         replace=True,
                         n_samples=25112,
                         random_state=0)
df_Romance   = resample(df_Romance,
                        replace=True,
                        n_samples=25112,
                        random_state=0)
df_Action  = resample(df_Action,
                      replace=True,
                      n_samples=25112,
                      random_state=0)
df_Horror     = resample(df_Horror, 
                         replace=True,     
                         n_samples=25112,    
                         random_state=0)
df_Documentary= resample(df_Documentary, 
                         replace=True,
                         n_samples=25112,    
                         random_state=0)
df_Crime   = resample(df_Crime,
                      replace=True,     
                      n_samples=25112,    
                      random_state=0)
df_Adventure= resample(df_Adventure, 
                       replace=True,  
                       n_samples=25112,    
                       random_state=0)
df_Mystery      = resample(df_Mystery,
                           replace=True,     
                           n_samples=25112,   
                           random_state=0)
df_Animation  = resample(df_Animation, 
                         replace=True, 
                         n_samples=25112, 
                         random_state=0)
df_Fantasy   = resample(df_Fantasy, 
                                 replace=True,     
                                 n_samples=25112,    
                                 random_state=0) 

# df_Family          = resample(df_Family, 
#                               replace=True,
#                               n_samples=25112,  
#                               random_state=0)
df_War       = resample(df_War, 
                        replace=True,   
                        n_samples=25112,    
                        random_state=0)
df_Western    = resample(df_Western, 
                         replace=True,     
                         n_samples=25112,    
                         random_state=0)
df_Music      = resample(df_Music,
                         replace=True,  
                         n_samples=25112, 
                         random_state=0)  
df_Children    = resample(df_Children, 
                         replace=True, 
                         n_samples=25112,  
                         random_state=0) 
df_SciFi   = resample(df_SciFi, 
                        replace=True,
                        n_samples=25112,  
                        random_state=0)
df_FilmNoir   = resample(df_FilmNoir, 
                        replace=True,
                        n_samples=25112,  
                        random_state=0)


df_IMAX = resample(df_IMAX, 
                       replace=True, 
                       n_samples=25112,
                       random_state=0)

In [143]:
df_upsampled = pd.concat([df_Drama, df_Comedy], axis=0 )
df_upsampled = pd.concat([df_upsampled, df_Thriller ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Action ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Adventure ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Romance ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Crime ]
                         )
df_upsampled = pd.concat([df_upsampled, df_SciFi ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Horror ]
                         ) 
df_upsampled = pd.concat([df_upsampled, df_Fantasy ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Mystery ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Animation ]
                         )  
df_upsampled = pd.concat([df_upsampled, df_Music ]
                         )
df_upsampled = pd.concat([df_upsampled, df_War ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Documentary ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Western ]
                         )  
df_upsampled = pd.concat([df_upsampled, df_Children ]
                         )
df_upsampled = pd.concat([df_upsampled, df_FilmNoir ]
                         )
df_upsampled = pd.concat([df_upsampled, df_IMAX ]
                         )

In [144]:
df_upsampled.reset_index()

,index,overview,genres
0,11,"Cheated on, mistreated and stepped on, the wom...",Drama
1,26,"Widowed U.S. president Andrew Shepherd, one of...",Drama
2,33,A look at President Richard M. Nixon—a man car...,Drama
3,38,"In early-1970s Las Vegas, low-level mobster Sa...",Drama
4,39,"Rich Mr. Dashwood dies, leaving his second wif...",Drama
...,...,...,...
477123,21176,"Harry starts his fourth year at Hogwarts, comp...",IMAX
477124,41824,"Jack Ryan, as a young covert CIA analyst, unco...",IMAX
477125,41994,"200 years after his shocking creation, Dr. Fra...",IMAX
477126,39985,"In 1942, Henan Province was devastated by the ...",IMAX


In [145]:
df_upsampled['genres'].value_counts()

Children       25112
Fantasy        25112
Animation      25112
Action         25112
Sci-Fi         25112
Horror         25112
Crime          25112
Thriller       25112
Mystery        25112
IMAX           25112
Drama          25112
War            25112
Film-Noir      25112
Musical        25112
Comedy         25112
Romance        25112
Adventure      25112
Western        25112
Documentary    25112
Name: genres, dtype: int64

In [146]:
# split the data, leave 1/3 out for testing
data_x = dados_dos_filmes[['overview']]
data_y = dados_dos_filmes.drop(['movieId', 'title', 'genres', 'overview'], axis=1)
# stratified_split = StratifiedShuffleSplit(n_splits=2, test_size=0.33)
xtrain, xval, ytrain, yval = model_selection.train_test_split(data_x, data_y, test_size=0.33, random_state=512)
# transform matrix of plots into lists to pass to a TfidfVectorizer
train_x = [x[0].strip() for x in xtrain.values.tolist()]
test_x = [x[0].strip() for x in xval.values.tolist()] 



In [147]:
from sklearn.ensemble import RandomForestClassifier
nltk.download('stopwords') 
words = stopwords.words("english") 
pipeline = Pipeline([
('tfidf', TfidfVectorizer(stop_words=words)),
('rfc', RandomForestClassifier() ),
    
])
parameters = {
# 'tfidf__max_df': (0.5, 0.65, 0.75),
# 'tfidf__min_df': (0.15, 0.30, 0.35),
'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
"rfc__n_estimators": [10,15,20],
"rfc__class_weight": ['balanced','balanced_subsample', None],
'rfc__max_depth':[20,25,30,35,40],
}

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

grid_search_tune = GridSearchCV(
pipeline, parameters, cv=2, n_jobs=2, verbose=6)
grid_search_tune.fit(train_x, ytrain )  
print
print("Best parameters set:")
print (grid_search_tune.best_estimator_.steps)

Fitting 2 folds for each of 135 candidates, totalling 270 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:  2.2min


In [ ]:

clf  = grid_search_tune.best_estimator_
# classes=categories
classes=[ 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
print('_' * 120)
print("Training: ")
print(clf)
t0 = time()  
train_time = time() - t0
print("Treino: %0.3fs" % train_time)

t0 = time()
pred = clf.predict(test_x)
test_time = time() - t0
print("testes:  %0.3fs" % test_time)

score = f1_score(yval, pred, average="micro")
precision = precision_score(yval, pred, average='micro')
print("f1_score:   %0.3f" % score)

print('\nAccuracy: {:.2f}\n'.format(accuracy_score(yval, pred)))
print('\nPrecisao: {:.2f}\n'.format(precision)) 

print("Relatório de Classificação:")
print(classification_report(yval, pred,
#                                             labels=[1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
                                        target_names=classes))

# print("confusion matrix:") 
# print(confusion_matrix(yvalb2.argmax(axis=1), pred.argmax(axis=1)))


if hasattr(clf, 'coef_'):
    print("dimensionality: %d" % clf.coef_.shape[1])
    print("density: %f" % density(clf.coef_))

clf_descr = 'RandomForest-upsampled-movielens'#str(clf).split('(')[0]

test_time=55555    
results.append([clf_descr, score,precision, train_time, test_time])  
resultsLogisticRegression.append([clf_descr, score,precision, train_time, test_time])  
salvar_dado([clf_descr, score,precision, train_time, test_time])


# Refazer predição com resample e IF-IDF

In [ ]:
df_upsampled = pd.concat([df_Drama, df_Comedy], axis=0 )
df_upsampled = pd.concat([df_upsampled, df_Thriller ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Action ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Adventure ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Romance ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Crime ]
                         )
df_upsampled = pd.concat([df_upsampled, df_SciFi ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Horror ]
                         ) 
df_upsampled = pd.concat([df_upsampled, df_Fantasy ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Mystery ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Animation ]
                         )  
df_upsampled = pd.concat([df_upsampled, df_Music ]
                         )
df_upsampled = pd.concat([df_upsampled, df_War ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Documentary ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Western ]
                         )  
df_upsampled = pd.concat([df_upsampled, df_Children ]
                         )
df_upsampled = pd.concat([df_upsampled, df_FilmNoir ]
                         )
df_upsampled = pd.concat([df_upsampled, df_IMAX ]
                         )

In [ ]:
df_upsampled.reset_index()

In [ ]:
df_upsampled['genres'] = df_upsampled['genres'].map(lambda x: x.split(','))
df_upsampled['genres'].head() 


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
multilabel_binarizer3 = MultiLabelBinarizer()
y =multilabel_binarizer3.fit_transform(df_upsampled['genres'])
tfidf_vectorizer3 = TfidfVectorizer(max_df=0.7, max_features=5000)
# from sklearn import model_selection
xtrain, xval, ytrain, yval = model_selection.train_test_split(df_upsampled['overview'], y, test_size=0.2, random_state=512)
xtrain_tfidf = tfidf_vectorizer3.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer3.transform(xval)
train_x = xtrain_tfidf
test_x = xval_tfidf

In [ ]:
xtrain_tfidf = tfidf_vectorizer3.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer3.transform(xval)
train_x = xtrain_tfidf
test_x = xval_tfidf
from sklearn.ensemble import RandomForestClassifier
nltk.download('stopwords') 
rfc = RandomForestClassifier()
words = stopwords.words("english") 
parameters = {'n_estimators':[10,15,20],  
             "class_weight": ['balanced','balanced_subsample', None],
             'max_depth':[3,5,8]}
# clf = GridSearchCV(svc, parameters, cv=2, n_jobs=2, verbose=3) 
 

In [ ]:


# classes=multilabel_binarizer3.classes_
classes=[ 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
print('_' * 120)


t0 = time()  
##############
print("Training: ")
grid_search_tune = GridSearchCV(
rfc, parameters, cv=2, n_jobs=2, verbose=3)
grid_search_tune.fit(train_x, ytrain )  

clf  = grid_search_tune.best_estimator_
print(clf)
print 
############################
train_time = time() - t0
print("Treino: %0.3fs" % train_time)

t0 = time()
pred = clf.predict(test_x)
test_time = time() - t0
print("testes:  %0.3fs" % test_time)

score = f1_score(yval, pred, average="micro")
precision = precision_score(yval, pred, average='micro') 
print("f1_score :   %0.3f" % score)

print('\nAccuracy: {:.2f}\n'.format(accuracy_score(yval, pred)))
print('\nPrecisao: {:.2f}\n'.format(precision)) 

print('Micro Precision: {:.2f}'.format(precision_score(yval, pred, average='micro')))
print('Macro Precision: {:.2f}'.format(precision_score(yval, pred, average='macro')))
print('Weighted Precision: {:.2f}'.format(precision_score(yval, pred, average='weighted')))


print('Micro F1-score: {:.2f}\n'.format(f1_score(yval, pred, average='micro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(yval, pred, average='macro'))) 
print('Weighted F1-score: {:.2f}'.format(f1_score(yval, pred, average='weighted')))

print('Macro Recall: {:.2f}'.format(recall_score(yval, pred, average='macro'))) 
print('Micro Recall: {:.2f}'.format(recall_score(yval, pred, average='micro')))
print('Weighted Recall: {:.2f}'.format(recall_score(yval, pred, average='weighted')))

print("Relatório de Classificação:")
print(classification_report(yval, pred,
#                                             labels=[1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
                                        target_names=classes))

# print("confusion matrix:") 
# print(confusion_matrix(yvalb2.argmax(axis=1), pred.argmax(axis=1)))


if hasattr(clf, 'coef_'):
    print("dimensionality: %d" % clf.coef_.shape[1])
    print("density: %f" % density(clf.coef_))

clf_descr = 'LogisticRegression-upsampled'#str(clf).split('(')[0]

test_time=55555    
results.append([clf_descr, score,precision, train_time, test_time])  
resultsLogisticRegression.append([clf_descr, score,precision, train_time, test_time])  
salvar_dado([clf_descr, score,precision, train_time, test_time])
